In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy.io
from sklearn.metrics import confusion_matrix
import anndata as ad
import re
from scepia.sc import (
    infer_motifs, 
    plot,
    ScepiaDataset
)

%matplotlib inline

sc.settings.verbosity = 3
sc.logging.print_versions()
sc.set_figure_params(dpi=150)

work_dir = "/path_to_working_directory/" 

#### Data locations & Other parameters

In [ ]:
## Data locations ##
raw_spliced_path = "path_to_data/seurat_sf_counts.mtx"
features_path = "path_to_data/sf_genes.tsv"
barcodes_path = "path_to_data/sf_barcodes.tsv"
scaled_spliced_path = "path_to_data/seurat_integrated_scaled_counts.csv"
metadata_path = "path_to_data/seurat_metadata.csv"
#HVG_path = "path_to_data/seurat_HVG.csv"
umap_path = "path_to_data/seurat_umap_embedding.csv"

## Parameters to set ##
pc_set = 20
cluster_ident = "cluster_annotation"

#### Loading data

In [ ]:
raw_spliced = scipy.io.mmread(raw_spliced_path)
scaled_spliced = np.asmatrix(pd.read_csv(scaled_spliced_path, index_col = 0))
metadata = pd.read_csv(metadata_path, index_col = 0)
umap_embedding = np.asmatrix(pd.read_csv(umap_path, index_col = 0))

In [ ]:
## Edits to make adata.var matrix
barcodedata = pd.read_csv(barcodes_path, sep = "\t", header = None)[0].to_list()
featuredata = pd.read_csv(features_path, sep = "\t", header = None)[0].to_list()
featuredata = pd.DataFrame(featuredata, index = featuredata)
featuredata.columns = ['gene']

## Create adata object from the Seurat object
First create a adata object with the raw count matrix of the Seurat object (perform normalization after), or directly use the normalized matrix from the Seurat object (this is the matrix without scaling).

In [ ]:
## Building the adata object, matrices from Seurat need Transposing before adding.
adata = ad.AnnData(X = raw_spliced.todense().T,
                   obs = metadata,
                   var = featuredata
                  )
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)
adata.raw = adata.copy()
adata.obsm["X_umap"] = umap_embedding

## Update adata with scaled data filtered on genes of interest
Since we are building from an Seurat object with integrated assay, this assay already consists of a subselection of HVG genes that were overlapping between the integrated datasets. 

Therefore the HVG in this case is the index of the scaled integrated dataset.

In [ ]:
adata.obs = adata.obs.astype(str)
HVG = pd.read_csv(scaled_spliced_path, index_col = 0).index
#adata.var_names_make_unique()

In [ ]:
## Generate dispersions_norm and highly_variable parameters, storing information on HVG selected in Seurat.
### Integrated Seurat datasets are already filtered for HVG! ###
adata.var['dispersions_norm'] = 0
adata.var['highly_variable'] = 0
adata.var['highly_variable'][adata.var['gene'].isin(HVG)] = 1
adata.var.head()

#### Filter dataset on genes of interest (HVG)

In [ ]:
adata = adata[:,HVG]
adata.X = scaled_spliced.T

In [ ]:
## Checking the structure of the data object
adata.var.shape

#### Run neighbors selection 

In [ ]:
### Integrated Seurat datasets are already filtered for HVG! ###
## Filter adata on HVG
#adata = adata[:, adata.var['highly_variable']]

In [ ]:
## SCEPIA: louvain clustering, n_neighbors in Seurat = 20
## Rerun neighbor selection
sc.pp.neighbors(adata, n_neighbors = 20, n_pcs= pc_set)
# sc.tl.louvain(adata)

In [ ]:
## Seurat objects may contain information on HVG selection, 
## store these variables from your Seurat object and save them as follows:
#adata.var['highly_variable'] = adata.var['vst.variable']
#adata.var['dispersions_norm'] = adata.var['vst.variance.standardized']
adata.obs['louvain'] = adata.obs[cluster_ident]
adata

#### Rename clusters

In [ ]:
# new_cluster_names = ['']
# adata.rename_categories('louvain', new_cluster_names)
# adata.obs['cluster_annotation'] = adata.obs['louvain']

In [ ]:
adata.obs['timepoint'] = pd.Categorical(adata.obs['timepoint'], categories = ["5W", "6W", "6.5W","7W", "9W", "10W", "13W", "15W", "17W", "20W", "22W", "23W", "24W", "25W"], ordered = True)

In [ ]:
sc.pl.umap(adata, color = 'louvain')

In [ ]:
sc.pl.umap(adata, color = ['cluster_annotation'])

In [ ]:
sc.pl.umap(adata, color = ['NKX2-5','TNNT2'])

#### Run SCEPIA analysis

In [ ]:
## Had to place a soft link of both the ENCODE.H3K27ac.human as well as Genome hg38 folder, in the current working dir.
# sd = ScepiaDataset("ENCODE.H3K27ac.human")

In [ ]:
## Genome hg38 needed for ENCODE.H3K27ac.human
#import genomepy
#genomepy.install_genome("hg8")

In [ ]:
infer_motifs(adata, dataset="ENCODE.H3K27ac.human", 
             n_top_genes=4000, 
             min_annotated = 1) #, 
             #max_cell_types = 100
             #)

In [ ]:
sc.pl.umap(adata, color="cell_annotation")

In [ ]:
sc.pl.umap(adata, color="cluster_annotation")

In [ ]:
labels = list(set(np.hstack((adata.obs["louvain"].astype(str).unique(), adata.obs["cell_annotation"].astype(str).unique()))))

cnf_matrix = pd.DataFrame(confusion_matrix(adata.obs["louvain"], adata.obs["cell_annotation"], labels=labels), index=labels, columns=labels)
cnf_matrix = cnf_matrix.loc[adata.obs["louvain"].astype(str).unique(), adata.obs["cell_annotation"].astype(str).unique()]
cnf_matrix = cnf_matrix.div(cnf_matrix.sum(1), axis=0)
cm = sns.clustermap(cnf_matrix.T, cmap="Reds", figsize=(12.5,12), yticklabels=True)

In [ ]:
adata.uns["scepia"]["correlation"].sort_values("p_adj").head(10)

In [ ]:
ax = plot(adata, n_anno=50)

## Subset correlation table on significant hits

In [ ]:
## Run for the significant hits from SCEPIA, the top per cluster:
FDR = 0.1
#p_sig = 0.05

column_names = ['motif','motif_activity','factor','abs_correlation','correlation','p_adj']
#selected_columns = ['motif','correlation','pval','p_adj']
selected_columns = ['motif','correlation','abs_correlation','pval','p_adj']
cluster_top = pd.DataFrame()

# Make a table with the significant scepia results
tmp = adata.uns['scepia']['correlation'][selected_columns]
tmp = tmp.sort_values('p_adj')
tmp = tmp[tmp['p_adj'] <= FDR]
#tmp = tmp[tmp['pval'] <= p_sig]
tmp = tmp[~tmp.index.duplicated(keep='first')]
tmp['factor'] = tmp.index

# ! Consider sorting on abs_correlation and then selecting first of duplicates
tmp.head()

In [ ]:
sc.pl.umap(adata, color="seurat_clusters", legend_loc = 'on data')

In [ ]:
sc.pl.umap(adata, color="louvain", legend_loc = 'on data')

In [ ]:
adata.write('object.scepia.h5ad')

In [ ]:
adata.raw[1:10,1:10]